# iSWAP Gate

*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline
This tutorial will demonstrate how to generate the fideliy-optimized pulse for the iSWAP gate using Quanlse. The outline of this tutorial is as follows:
- Introduction
- Preparation
- Construct Hamiltonian
- Generate and optimize pulse via Quanlse Cloud Service
- Summary

## Introduction
The iSWAP gate swaps the excitations between two qubits and adds a phase of $i$ \[1\].
The corresponding unitary matrix is:
$$
\rm iSWAP = \begin{pmatrix}
    1 & 0 & 0 & 0 \\
    0 & 0 & -i & 0 \\
    0 & -i & 0 & 0 \\
    0 & 0 & 0 & 1   
\end{pmatrix}.
$$
The iSWAP gate is a native gate in superconducting quantum computing platforms, because it can be realized directly using coupling between two superconducting qubits (XY interaction). In superconducting circuits, the iSWAP gate is implemented by tuning the qubits into resonance in a duration of time. By performing a rotating wave approximation, the two-qubit coupling Hamiltonian can be written as \[2\]:
$$
\hat H_{\rm coup} = g(e^{i\delta_{01}t}\hat{a}_0^\dagger\hat{a}_1+e^{-i\delta_{01}t}\hat{a}_0\hat{a}_1^\dagger),
$$
where $\delta_{01}=\omega_{q0}-\omega_{q1}$ is the detuning. By tuning the frequency of qubit 0 into resonance with qubit 1, the detuning $\delta_{01}$ reduces to $0$; The truncated operators $a_i^\dagger$ and $a_i$ are creation operator and annihilation operator of qubit $i$. The coupling Hamiltonian reduces to:
$$
\hat H_{\rm coup} = g(\hat{a}_0^\dagger\hat{a}_1+\hat{a}_0\hat{\sigma}_1^\dagger),
$$
which swaps the excitations of the two qubits.

Moreover, two iSWAP gates and several single-qubit gates can be used to generate a CNOT gate. The generation of GHZ states utilizing the iSWAP gates has also been demonstrated \[2\].

## Preparation
After you have successfully installed Quanlse, you could run the Quanlse program below following this tutorial. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
# Import numpy and math constants
from numpy import round
from math import pi

# Import the Hamiltonian module
from Quanlse.QHamiltonian import QHamiltonian as QHam
from Quanlse.QOperator import duff, number

# Import Optimizer for ISWAP gate
from Quanlse.remoteOptimizer import remoteOptimizeISWAP

# Import tools for analysing results
from Quanlse.Utils.Functions import project

## Construct Hamiltonian

To perform optimization using Quanlse, we need to specify the system Hamiltonian. We consider the three lowest energy levels of each qubit where the third level accounts for leakage. In the rotating frame, the system Hamiltonian of two coupled qubits is expressed as \[1\]:
$$
\hat H =(\omega_{q0}-\omega_{d0})(\hat a^\dagger_0 \hat a_0)+(\omega_{q1}-\omega_{d1})(\hat a^\dagger_1 \hat a_1)+ \frac{\alpha _0}{2}\hat a^\dagger_0 \hat a^\dagger_0 \hat a_0 \hat a_0 + \frac{\alpha_1}{2}\hat a^\dagger_1 \hat a^\dagger_1 \hat a_1 \hat a_1 +\frac{g}{2}(\hat a_0\hat a_1^\dagger+\hat a_0^\dagger \hat a_1) + \sum_i\frac{A^z_i(t)}{2}\hat a^\dagger_i \hat a_i,
$$
where $\hat a_i^\dagger$ ($\hat a_i$) is the creation (annihilation) operator of qubit $i$ ($i = 0,1$); $\omega _{qi}$ is the frequency of qubit $i$; $\alpha_i$ is the anharmonicity of qubit $i$; and $g$ is the coupling strength between the qubits. $A_i^z(t)$ is the magnetic flux applying on qubit $i$.

Quanlse defines the system Hamiltonian by specifying the sampling period, gate time and everything else necessary to set up the calculation. Quanlse provides the flexibility to customize simulation for practically any gate operations, particularly those involving parameter optimization. First, we define a couple of parameters to set the sampling period, the number of qubits, and the system dimension: 

In [ ]:
# Sampling period
dt = 1.0
# Number of qubits
qubits = 2
# System energy level
level = 3

We then define the transition frequencies, drive frequencies, and anharmonicities of the two qubits and the coupling strength in between:

In [ ]:
qubitArgs = {
    "coupling": 0.0277 * (2 * pi),  # Coupling of Q0 and Q1
    "qubit_freq0": 5.805 * (2 * pi),  # Frequency of Q0
    "qubit_freq1": 5.205 * (2 * pi),  # Frequency of Q1
    "drive_freq0": 5.205 * (2 * pi),  # Drive frequency on Q0
    "drive_freq1": 5.205 * (2 * pi),  # Drive frequency on Q1
    "qubit_anharm0": -0.217 * (2 * pi),  # Anharmonicity of Q0
    "qubit_anharm1": -0.226 * (2 * pi)  # Anharmonicity of Q1
}

Then, we can proceed to create an empty Hamiltonian dictionary using the `QHamiltonian()` function and add terms to it based on previously defined parameters:

$$
\begin{align*}
\hat H_{\rm drift} &= (\omega_{q0}-\omega_{d0})(\hat a^\dagger_0 \hat a_0)+(\omega_{q1}-\omega_{d1})(\hat a^\dagger_1 \hat a_1)+\frac{\alpha _0}{2}\hat a^\dagger_0 \hat a^\dagger_0 \hat a_0 \hat a_0 + \frac{\alpha_1}{2}\hat a^\dagger_1 \hat a^\dagger_1 \hat a_1 \hat a_1, \\
\hat H_{\rm coup} &= \frac{g}{2}(\hat a_0\hat a_1^\dagger+\hat a_0^\dagger \hat a_1).
\end{align*}
$$

In Quanlse's `QOperator` module, we have provided tools that would allow the users to construct the commonly used operators quickly. The detuning term $(\omega_{\rm q}-\omega_{\rm d})\hat{a}^\dagger \hat{a}$ and the anharmonicity term $\frac{\alpha}{2}\hat{a}^\dagger\hat{a}^\dagger \hat{a} \hat{a} $ can be respectively generated using  `number(n)` and `duff(n)` from the `QOperator` module: the two functions `number(n)` and `duff(n)` return the $n \times n$ matrices for number operators and duffing operators. Thus we can directly add these terms to the Hamiltonian by using function `addDrift()`. The coupling term, which takes the form, $\frac{g}{2}(\hat{a}_i^\dagger\hat{a}_j+\hat{a}_i\hat{a}_j^\dagger$), can be directly added to the Hamiltonian using function `addCoupling()`.

In [ ]:
# Create the Hamiltonian
ham = QHam(qubits, level, dt)

# Add the drift term(s)
for qu in range(2):
    # Add the detuning term(s)
    ham.addDrift(number, qu, (qubitArgs[f"qubit_freq{qu}"] - qubitArgs[f"drive_freq{qu}"]))
    # Add the anharmonicity term(s)
    ham.addDrift(duff, qu, qubitArgs[f"qubit_anharm{qu}"] / 2)

# Add coupling term
ham.addCoupling([0, 1], qubitArgs["coupling"] / 2)

Noted that the optimizer in Quanlse will automatically add the control term:

$$ 
\hat{H}_{\rm ctrl} = \frac{A_0^z(t)}{2}\hat{a}_0^\dagger \hat{a}_0, 
$$

thus we don't need to add this term manually.

All useful information regarding the system Hamiltonian is stored in `ham`. We should set a boundary for amplitudes before launching the optimization. You might need to run the optimization several times and narrow down the boundary gradually.

In [ ]:
aBound = (-5, 5)

## Generate and optimize pulse via Quanlse Cloud Service

The optimization process usually takes a long time to process on local devices, however, we provide a cloud service that could speed up this process significantly. To use the Quanlse Cloud Service, the users need to acquire a token from http://quantum-hub.baidu.com.

In [ ]:
# Import tools to get access to cloud service
from Quanlse import Define

# To use remoteOptimizerISWAP() on cloud, paste your token (a string) here
Define.hubToken = ''

Now, we are ready to run the optimization. The `remoteOptimizeISWAP()` function takes five arguments. `tg` specifies the gate time. When `targetInfidelity` is reached, or the number of iterations exceeds `maxIter`, the optimization will terminate and return the minimum infidelity and the system Hamiltonian with optimized control terms. We can plot the pulses by calling `qham.plotWaves()` and get the unitary evolution by calling `qham.getUnitary()`. 

The gate infidelity for performance assessment throughout this tutorial is defined as ${\rm infid} = 1 - \frac{1}{d}\left|{\rm Tr}\left[U^\dagger_{\rm goal}P(U)\right]\right|$, where $d$ is the dimension of $U_{\rm goal}$ $(U_{\rm goal} = {\rm iSWAP})$ and $U$ is the unitary evolution of the three-level system. Note that $P(U)$ in particular describes the evolution projected to the computational subspace. We can first  run the optimization then calculate the projected evolution $P(U)$ :

In [ ]:
# Run the optimization
gateJob, infidelity = remoteOptimizeISWAP(ham, aBound=aBound, tg=40, maxIter=3, targetInfidelity=0.005)

# Print infidelity and the waveforms
print(f"minimum infidelity: {infidelity}")
gateJob.plot()

# Print the projected evolution
result = ham.simulate(job=gateJob)
print("The projected evolution P(U):\n", round(project(result[0]["unitary"], qubits, level, 2), 2))

We can see that the drive strength of the pulse is around 3.8. We can zoom in and search for a better fidelity by narrowing down the boundary to (-4.0, -3.6).

In [ ]:
aBound = (-4.0, -3.6)
# Run the optimization
gateJob, infidelity = remoteOptimizeISWAP(ham, aBound, tg=40, maxIter=5, targetInfidelity=0.01)

# Print infidelity and the waveforms
print(f"minimum infidelity: {infidelity}")
gateJob.plot()

# Print the projected evolution 
result = ham.simulate(job=gateJob)
print("The projected evolution P(U):\n", round(project(result[0]["unitary"], qubits, level, 2), 2))

If you wish to further reduce the infidelity, try setting the parameter `maxIter` to a larger number. With a set of reasonable parameters, Quanlse can find the optimal waveforms.

## Summary
This tutorial introduces the pulse optimization of the iSWAP gate using Quanlse. After reading this tutorial, the users could follow this link [tutorial-iswap](https://github.com/baidu/Quanlse/blob/main/Tutorial/EN/tutorial-iswap.ipynb) to the GitHub page of this Jupyter Notebook document and run this program for themselves. The users are encouraged to try parameter values different from this tutorial to obtain the optimal result.

## References
\[1\] [Schuch, Norbert, and Jens Siewert. "Natural two-qubit gate for quantum computation using the XY interaction." *Physical Review A* 67.3 (2003): 032301.](https://link.aps.org/doi/10.1103/PhysRevA.67.032301)

\[2\] [Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." *Applied Physics Reviews* 6.2 (2019): 021318.](https://doi.org/10.1063/1.5089550)